# Beampattern Notebook

Concepts to communicate:

1. Beampattern
2. Main Lobe
3. Sidelobes
4. Beam width
5. Source Level
6. Near-field vs far-field

A **transducer** is a device that takes an electrical signal and transforms it into mechanical motion. The audio speakers in calculaters, phones, or headphones are probably the transducers we interact with and use on a daily basis. These transducers use an electrical signal to cause the speaker surface to oscillate which then generates sound. The acoustic source in a sonar system does the same thing. A transducer can also take mechanical motion and transform it into an electrical signal (microphones in air and hydrophones in the water), but this section will focus only on the properties of acoustic transducers that are used to generate sound.

## Circular transducer

The most common man-made sound sources used in underwater acoustics are planer transducers which have flat surfaces which oscillate to produce sound. This is in part due to the fact that a flat transducer is relatively easy to manufacturer, but more importantly, the transducer transmits a majority of the sound in the direction perpendicular to the vibrating plane.  This is a useful property in many sonar systems and this directivity of the transmitted sound can help determine the location of an object (fish, bubble, submarine, etc.)

Note that we said the transducer directs a "majority" of the sound in one direction and not "all" of the sound. In fact, the transducer sends different levels of sound in many directions and the directional dependence of the sound is refered to as the transducer's **beam pattern**. To help understand the concept of a beam pattern and it's properties, we will focus initially on one particular, common type of tranducer: the circular piston. This type of transducer is often a cylindrical piece of piezoelectric material to which an electrical signal is applied such that it expands and contracts along the axis of the cylinder. The causes sound to radiate from the circular face of the transducer. The structure of the sound field from this type of transducer can be fairly complicated and there is not a simple, mathematical expression for the direction and strength of the transmitted sound. There are, however, approximate mathematical descriptions that can be accurately describe the transmitted sound under certain conditions. One of the most useful of these is to assume that the transducer an be approximated as a baffled circular piston. By "baffled," we mean that the face of the transducer is surrounded by an infinite, rigid surface. This simplifies the mathematics and allows us to calculate strength and direction of the transmitted field.

In the following, the face of the transducer will be in the x-y plane and the z-axis is perpendicular to the face. The radius of the tranducer is ***a*** and the field will be described in two ways; either a function of the distance, ***r***, from the center of the face and the angle, ***$\theta$***, between the direction of ***r*** and the -axis or as a function of distance from the plane of the transducer (range) and the perpendicular direction (cross-range.)

![Piston](images/quick_piston.png)

 If the sound is measured far away from the face of the transducer, the intensity of the sound will be proportional to $1/r^2$ and the angular-dependence of the field can be expressed in a simple form that does not depend on the distance. When the reciever is sufficiently far away that these two properties are true, then the receiver is said to be in the **far-field** of the transducer. In the widget below, we've removed the $1/r^2$ dependence and normalized the intensity by the value measured on the z-axis or $\theta = 0^\circ$. As a result, the normalized intensity is 0 dB (or 1 on a linear scale.) The widget allows you to make the transducer larger or smaller by increasing it's radius and allows you to change the frequency of the sound being transmitted. You can also display the angular dependence of the transmitted sound on different plots and on both a linear and decibel scale.

In [1]:
# Import packages
import multiprocessing as mp
from functools import partial

import numpy as np
import matplotlib.pyplot as plt
from scipy.special import j1  # Bessel function of the first kind
import ipywidgets as widgets
from ipywidgets import interact
import pandas as pd

In [7]:
# Constants
rho0 = 1
c = 1500
r = 10

# Create sliders for 'f' and 'a'
f_slider = widgets.IntSlider(min=1, max=50, step=1, value=1, description='Frequency (kHz)')
a_slider = widgets.FloatSlider(min=0.1, max=1, step=0.05, value=0.5, description='Aperture Radius (m)')

def far_field_intensity_cartesian_polar_calculation(f, a):
    # Calculate angular frequency
    omega = 2 * np.pi * f * 1000

    # Calculate wavenumber
    k = omega / c

    # Projection Intensity calculation
    theta = np.arange(-90, 90.1, 0.1)
    u = k * a * np.sin(theta * np.pi / 180)
    projInt = np.abs((rho0 * c * k * a * a / (2 * r)) * 
                     (2 * j1(u) / u)) ** 2

    # Handle division by zero at theta = 0
    ind = np.where(theta == 0)[0]
    axisInt = np.abs(rho0 * c * k * a * a / (2 * r)) ** 2
    projInt[ind] = axisInt

    return theta, projInt, axisInt

def update_far_field_intensity_cartesian_polar_plot(f, a, plotType, plotScale):
    # Calculate far field intensity
    theta, projInt, axisInt = far_field_intensity_cartesian_polar_calculation(f, a)

    # Close any existing plots
    plt.close('all') 

    # Plot in Cartesian
    if plotType == 'Cartesian':
        if plotScale == "Decibels":
            plt.plot(theta, 10 * np.log10(projInt / axisInt), linewidth=2)
            plt.ylim([-40, 0])
            plt.xlim([-90, 90])
            plt.xlabel('Angle (degrees)', fontsize=14)
            plt.ylabel('Beam Level (dB)', fontsize=14)
            plt.grid(True)
        else:
            plt.plot(theta, projInt / axisInt, linewidth=2)
            plt.ylim(10 ** (np.array([-40, 0]) / 10))
            plt.xlim([-90, 90])
            plt.xlabel('Angle (degrees)', fontsize=14)
            plt.ylabel('Beam Level', fontsize=14)
            plt.grid(True)

    # Plot in Polar
    elif plotType == 'Polar':
        if plotScale == "Decibels":
            plt.polar(theta * np.pi / 180, 10 * np.log10(projInt / axisInt), linewidth=2)
            ax = plt.gca()
            ax.set_ylim([-40, 0])
            ax.set_theta_zero_location('N')  # 'top'
            ax.set_thetalim([np.radians(-90), np.radians(90)])
            ax.set_title('Beam Pattern (dB)', fontsize=14)
        else:
            plt.polar(theta * np.pi / 180, projInt / axisInt, linewidth=2)
            ax = plt.gca()
            ax.set_ylim(10 ** (np.array([-40, 0]) / 10))
            ax.set_theta_zero_location('N')  # 'top'
            ax.set_thetalim([np.radians(-90), np.radians(90)])
            ax.set_title('Beam Pattern (Linear Scale)', fontsize=14)

    # Calculate beam widths
    ind3dB = np.where(10 * np.log10(projInt / axisInt) >= -3)[0]
    ind6dB = np.where(10 * np.log10(projInt / axisInt) >= -6)[0]
    ind10dB = np.where(10 * np.log10(projInt / axisInt) >= -10)[0]
    bw3dB = np.abs(theta[ind3dB[0]])
    bw6dB = np.abs(theta[ind6dB[0]])
    bw10dB = np.abs(theta[ind10dB[0]])

    # Define labels and values
    bwLabels = ['down 3 dB', 'down 6 dB', 'down 10 dB']
    bwValues = [bw3dB, bw6dB, bw10dB]

    # Create a DataFrame
    df = pd.DataFrame({
        'Beam Width (degrees)': bwValues,
        'Definition': bwLabels
    })

    # Print the DataFrame
    print(df)

# Create widgets for plot type and scale
plot_type_widget = widgets.Dropdown(
    options=['Cartesian', 'Polar'],
    value='Polar',
    description='Plot Type:'
)

plot_scale_widget = widgets.Dropdown(
    options=['Decibels', 'Linear'],
    value='Decibels',
    description='Scale:'
)

# Use interact to link widgets and plotting function
interact(
    update_far_field_intensity_cartesian_polar_plot,
    f=f_slider,
    a=a_slider,
    plotType=plot_type_widget,
    plotScale=plot_scale_widget,
)

interactive(children=(IntSlider(value=1, description='Frequency (kHz)', max=50, min=1), FloatSlider(value=0.5,…

<function __main__.update_far_field_intensity_cartesian_polar_plot(f, a, plotType, plotScale)>

One of the things you'll notice as you decrease the frequency or decrease the radius, is that the central part of the beam pattern gets wider. This central portion of the beam pattern that includes the maximum at 0 degrees and is enclosed by the first null (points where the level gets very low) is called the **main lobe or beam**. Most of the energy of the transmitted sound lies in this beam. As a result, most applications try to utilize just this beam. A key parameter for the transducer is width of this main beam and is referred to as the **beam width**. The beam width is defined as the width of the beam when the field drops to a set level below the maximum value. This level is sometimes defined as 3 dB, 6 dB, or 10 dB which on a linear scale correspond to 1/2, 1/4, and 1/10 of the maximum of the maximum value of the main lobe. For example, [SS510 echosounder spec sheet](https://www.teledynemarine.com/en-us/products/SiteAssets/Odom/transducers/SMBB200-9.pdf) states that when operating at 200 kHz, the beam width is 9 degrees @ -3 dB. The table above shows the beamwidth defined relate to the different levels for the plotted beam pattern.

The other beams or lobes are often referred to as **side lobes**. Notice that while the shape of the main lobe and the side lobes change significantly with frequency and transducer radius, the maximum level of the two sidelobes on either side of the main lobe is always at the same level, 17 dB, or 1/50 of the main lobe. This doesn't always hold true for all piston transducers but is a good approximation for the level of the first side lobe and is a useful rule of thumb for the far-field of the transducer. 

In [8]:
# Constants
rho0 = 1
c = 1500

# Create sliders for 'f' and 'a'
f_slider = widgets.IntSlider(min=1, max=50, step=1, value=1, description='Frequency (kHz)')
a_slider = widgets.FloatSlider(min=0.1, max=1, step=0.05, value=0.5, description='Aperture Radius (m)')

def far_field_intensity_x_y_plane_calculation(f, a, r, cr):
    # Calculate angular frequency
    omega = 2 * np.pi * f * 1000

    # Calculate wavenumber
    k = omega / c

    # Calculate sound pressure level in linear units
    SL_dB = 180
    SL_lin = 10**(SL_dB / 20)

    # Calculate the particle velocity amplitude
    U0 = SL_lin / (rho0 * c * k * a * a / 2)

    # Calculate meshgrids
    x = np.linspace(-cr, cr, 150)
    y = np.linspace(0, r, 100)
    X, Y = np.meshgrid(x, y)

    # Far field calculation
    thetaXY = np.arctan2(X, Y)
    rXY = np.sqrt(X**2 + Y**2)
    u1 = k * a * np.sin(thetaXY)
    u1[u1 == 0] = np.nan  # Avoid division by zero
    projIntXY = np.abs((rho0 * c * k * U0 * a * a / (2 * rXY)) *
                    (2 * j1(u1) / u1))**2

    return x, y, SL_dB, projIntXY

def update_far_field_intensity_x_y_plane_plot(f, a, plotScale):
    # Calculate far field intensity on x-y plane
    x, y, SL_dB, projIntXY = far_field_intensity_x_y_plane_calculation(f, a, r=10, cr=10)

    # Close any existing plots
    plt.close('all')

    # Plotting
    plt.figure(1)
    if plotScale == 'Decibels':
        plt.imshow(10 * np.log10(projIntXY), extent=(x.min(), x.max(), y.min(), y.max()), origin='lower')
        plt.colorbar(label='Intensity (dB re 1μPa)')
        plt.clim(SL_dB - 90, SL_dB)
    else:
        plt.imshow(projIntXY / 1e6, extent=(x.min(), x.max(), y.min(), y.max()), origin='lower')
        plt.colorbar(label='Intensity (μPa²)')
        plt.clim(10**((SL_dB - 90) / 10), SL_dB)
        
    plt.xlabel('Cross Range (m)')
    plt.ylabel('Range (m)')
    plt.gca().set_aspect('auto')
    plt.show()

# Use interact to link widgets and plotting function
interact(
    update_far_field_intensity_x_y_plane_plot,
    f=f_slider,
    a=a_slider,
    plotScale=plot_scale_widget,
)

interactive(children=(IntSlider(value=1, description='Frequency (kHz)', max=50, min=1), FloatSlider(value=0.5,…

<function __main__.update_far_field_intensity_x_y_plane_plot(f, a, plotScale)>

## Far-field approximation compared to the full field

One important property of the far-field of a transducer is that the sound in this region all appears to coming from a single point on the transducer. This is what leads to the  $1/r^2$ decrease in the intensity with range. Near the face of the transducer, this is no longer true and there is a region where the sound from one part of the transducer is interfering with sound from other parts of the transducer and there are no clear beams. This region is refered to as the **near-field** of the transducer. There is no simple mathematical expression for the sound field in this region, so in the widget below the sound field is calculated numerically to compare with the far-field solution. In the upper left plot is the far-field approximation of the sound field where regardless of how close you get to the transducer face, the field will always appear to come from a point at the center of the transducer. In the upper right plot, the true field of the transducer is numerically calculated. If the plotted range and cross-range is large, the two plots will look very similar. However if the range and cross-range start to get close to the radius of the transducer, the true field becomes very complicated.  The lower plot shows the field along the main axis of the transducer. Far from the face, the far-field and numerical calculation are very similar. Close to the transducer, the field begins to oscillate with strong nulls and peaks. There are a number of ways to estimate when this transition between near-field and far-field occurs. One useful estimate is that the range from the transducer needs to satisfy $r\gg2a$  and $r \gg \pi a^2/\lambda$. In these calculations, the speed of sound is $c = 1500$ m/s and using the relationship $c = f\lambda$ which relates the sound speed, frequency $f$, and wavelength, $\lambda$, you can varify that this conditions hold as you change the frequency and radius of the transducer. 

In [ ]:
# Constants
rho0 = 1
c = 1500

# Create sliders for 'f' and 'a'
f_slider = widgets.IntSlider(min=1, max=50, step=1, value=1, description='Frequency (kHz)')
a_slider = widgets.FloatSlider(min=0.1, max=1, step=0.1, value=0.5, description='Aperture Radius (m)')
r_slider = widgets.IntSlider(min=1, max=10, step=1, value=10, description='Range (m)')
cr_slider = widgets.IntSlider(min=1, max=10, step=1, value=10, description='Cross Range (m)')

def calculate_x_chunk_intValXY1(jX, y1, rXY1, sigmaP, dsigmaP, thetaXY1, dphiP, k, phiP):
    result = np.zeros_like(y1, dtype=np.complex128)
    for jY in range(len(y1)):
        rPXY1 = np.sqrt(
            np.square(rXY1[jY, jX]) + np.square(sigmaP) + (2*rXY1[jY, jX]*sigmaP)*np.sin(thetaXY1[jY, jX])*(np.cos(thetaXY1[jY, jX])*np.cos(phiP.reshape(-1,1)) + np.sin(thetaXY1[jY, jX])*np.sin(phiP.reshape(-1,1)))
        )
        result[jY] = np.sum(np.sum(sigmaP * dsigmaP * dphiP * np.exp(-1j * k * rPXY1) / rPXY1))
    return result

def calculate_intValXY1_multiprocessing(x1,y1, rXY1, sigmaP, dsigmaP, thetaXY1, dphiP, k, phiP):
    num_processes = mp.cpu_count()
    with mp.Pool(num_processes) as pool:
        # Create a partial function with fixed additional arguments
        partial_func = partial(calculate_x_chunk_intValXY1, y1=y1, rXY1=rXY1, sigmaP=sigmaP, dsigmaP=dsigmaP, thetaXY1=thetaXY1, dphiP=dphiP, k=k, phiP=phiP)
        intValXY1 = pool.map(partial_func, range(len(x1)))

    return np.array(intValXY1).T

def far_and_full_field_intensity_x_y_plane_calculation(f, a, r, cr):
    # Calculate angular frequency
    omega = 2 * np.pi * f * 1000

    # Calculate wavenumber
    k = omega / c

    # Calculate the wavelength
    wavelength = c / (f * 1000)

    # Calculate the boundary distance for near field to far field
    boundary_distance = 2 * a**2 / wavelength

    # Calculate sound pressure level in linear units
    SL_dB = 180
    SL_lin = 10**(SL_dB / 20)

    # Calculate the particle velocity amplitude
    U0 = SL_lin / (rho0 * c * k * a * a / 2)

    # Create mesh grids
    x1 = np.linspace(-cr, 0, 50)
    x1 = np.concatenate((x1, -x1[-2::-1]))
    y1 = np.linspace(0, r, 200)
    X1, Y1 = np.meshgrid(x1, y1)

    ### Far Field Calculations:

    # Calculate thetaXY1, rXY1, and u11
    thetaXY1 = np.arctan2(X1, Y1)
    rXY1 = np.sqrt(X1**2 + Y1**2)
    rXY1[rXY1 == 0.0] = 1e-10
    u11 = k * a * np.sin(thetaXY1)
    u11[u11 == 0.0] = 1e-10
    projIntXY1 = np.square(np.abs((rho0 * c * k * U0 * np.square(a) / (2 * rXY1)) * (2 * j1(u11) / u11)))
    indX1 = np.argmin(np.abs(x1))
    projIntXY1[:, indX1] = np.square(np.abs(rho0 * c * U0 * k * np.square(a) / (2 * rXY1[:, indX1])))

    ### Full Field Calculations:

    # Create sigmaP and phiP
    sigmaP = np.linspace(0, a, 200)
    phiP = np.linspace(0, 2 * np.pi, 200)
    dsigmaP = np.diff(sigmaP[:2])
    dphiP = np.diff(phiP[:2])
    
    # Calculate intValXY1 using multiprocessing since it's a large integral to calculate
    intValXY1 = calculate_intValXY1_multiprocessing(x1, y1, rXY1, sigmaP, dsigmaP, thetaXY1, dphiP, k, phiP)

    # Calculate nfprojIntXY
    nfprojIntXY = np.square(np.abs(1j * rho0 * c * U0 * k / (2 * np.pi) * intValXY1))

    return boundary_distance, x1, y1, indX1, SL_dB, projIntXY1, nfprojIntXY

def update_far_and_full_field_intensity_x_y_plane_plot(f, a, r, cr):
    # Calculate far and full field intensities on the x-y plane
    boundary_distance, x1, y1, indX1, SL_dB, projIntXY1, nfprojIntXY = far_and_full_field_intensity_x_y_plane_calculation(f, a, r, cr)

    # Close any existing plots
    plt.close('all')

    # Create subplots
    _, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(19, 6))

    # Convert projIntXY1 and nfprojIntXY to dB scale
    projIntXY1_dB = 10 * np.log10(projIntXY1)
    nfprojIntXY_dB = 10 * np.log10(nfprojIntXY)

    # Far field x-y plane intensity plot
    im1 = ax1.imshow(projIntXY1_dB, extent=(x1.min(), x1.max(), y1.min(), y1.max()), origin='lower')
    ax1.set_title("Far Field Intensity")
    ax1.set_xlabel('Cross Range (m)')
    ax1.set_ylabel('Range (m)')
    cbar1 = plt.colorbar(im1, ax=ax1, label='Intensity (dB re 1μPa)')
    cbar1.ax.yaxis.set_label_position('right')
    cbar1.ax.yaxis.label.set_rotation(270)
    cbar1.ax.yaxis.labelpad = 13
    im1.set_clim(SL_dB - 90, SL_dB)
    ax1.set_aspect('auto')

    # Full field x-y plane intensity plot
    im2 = ax2.imshow(nfprojIntXY_dB, extent=(x1.min(), x1.max(), y1.min(), y1.max()), origin='lower')
    ax2.set_title("Full Field Intensity")
    ax2.set_xlabel('Cross Range (m)')
    ax2.set_ylabel('Range (m)')
    cbar2 = plt.colorbar(im2, ax=ax2, label='Intensity (dB re 1μPa)')
    cbar2.ax.yaxis.set_label_position('right')
    cbar2.ax.yaxis.label.set_rotation(270)
    cbar2.ax.yaxis.labelpad = 13
    im2.set_clim(SL_dB - 90, SL_dB)
    ax2.set_aspect('auto')

    # Far vs Full Field Intensity Comparison at points where Theta is 0
    ax3.plot(y1, projIntXY1_dB[:, indX1], label='Far Field where Theta == 0', linewidth=2, color="b")
    ax3.plot(y1, nfprojIntXY_dB[:, indX1], label='Full Field where Theta == 0', linewidth=2, color="r")
    ax3.set_title("Far and Full Field Intensity when Theta == 0")
    ax3.set_xlabel('Range (m)')
    ax3.set_ylabel('Intensity (dB re 1μPa)')
    ax3.yaxis.set_label_position('right')
    ax3.yaxis.label.set_rotation(270)
    ax3.yaxis.labelpad = 13
    ax3.yaxis.tick_right()
    ax3.set_aspect('auto')
    ax3.grid(True)
    # Plot the boundary line
    ax3.axvline(x=boundary_distance, color='k', linestyle='--', label='Near Field / Far Field Boundary')
    ax3.legend()

    plt.show()

# Use interact to link widgets and plotting function
interact(
    update_far_and_full_field_intensity_x_y_plane_plot,
    f=f_slider,
    a=a_slider,
    r=r_slider,
    cr=cr_slider,
)

interactive(children=(IntSlider(value=1, description='Frequency (kHz)', max=50, min=1), FloatSlider(value=0.5,…

## Rectangular Transducer